This assignment explores the venues in the city of Toronto with the postal code beginning with M.

Project Title: **Which location to open a health food store?**

**Introduction**: As people are paying increasing attention to their health. There is an increasing demand in health food,whole food and superfood. Finding the proper location to open a health food store is important for business success. This assignment help to solve this problem by using Foursquare venue data in the city of Toronto. 
Each neighborhood will be explored. The top 3 frequent venues will be selected and analyzed to find out which location is proper to open a health food store.

**Data**:
City of Toronto borough, neighborhood, postal codes were collected from "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M".
Venues data of Toronto was collected using Foursquare.
Toronto neighborhoods population size were collected from "https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods"

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    lib

In [2]:
df=pd.read_csv('week3 longitude and altitude.csv')

In [3]:
borough=pd.value_counts(df['BOROUGH'])
borough

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           11
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Queen's Park         1
Name: BOROUGH, dtype: int64

In [4]:
borough.shape

(11,)

In [5]:
neighborhood=pd.value_counts(df['NEIGHBORHOOD'])
neighborhood.shape

(102,)

There are 11 boroughs and 102 neighborhoods in the city of Toronto.

To create a map of city of Toronto including 11 boroughs

In [6]:
latitude = 43.6532
longitude= -79.3832
map_toronto= folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['NEIGHBORHOOD']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [7]:
CLIENT_ID = 'N043PBNGAO2F3G1AZ1UB1ICLOCFZMGCXO0RYQTA0WMNUBFNU' 
CLIENT_SECRET = 'SX4CB1W2ZDDX3SIEHUJZJYYBM5XJJP4412QUZTKDNC1VPMBK' 
VERSION = '20180605' 

In [8]:
LIMIT = 100 
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
       
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
venues = getNearbyVenues(names=df['NEIGHBORHOOD'],
                         latitudes=df['Latitude'],
                         longitudes=df['Longitude'])

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [10]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56


In [11]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 268 uniques categories.


Analyze each neighborhood in the city of Toronto

In [12]:
toronto_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [13]:
toronto_onehot.shape

(2215, 268)

calculate the frequency of each categories of venue

In [14]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000

List the top 10 venues in each neighborhood

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Asian Restaurant,Restaurant,Breakfast Spot,Hotel,Thai Restaurant,Gastropub
1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Coffee Shop,Pizza Place,Fried Chicken Joint,Sandwich Place,Beer Store,Fast Food Restaurant,Video Store,Pharmacy,Liquor Store
4,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Pharmacy,Skating Rink,Athletics & Sports,Pub,Sandwich Place,Discount Store,Dessert Shop
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Middle Eastern Restaurant,Frozen Yogurt Shop,Sandwich Place,Bridal Shop,Diner,Restaurant,Ice Cream Shop,Deli / Bodega,Supermarket
6,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Sandwich Place,Pizza Place,Restaurant,Sushi Restaurant,Café,Pub,Indian Restaurant
8,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Steakhouse,Beer Bar,Bakery,Café,Cheese Shop,Eastern European Restaurant
9,"Birch Cliff, Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [17]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [18]:
toronto_merged =df

toronto_merged =toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='NEIGHBORHOOD')

toronto_merged

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Fast Food Restaurant,Department Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Intersection,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Mexican Restaurant,Medical Center,Doner Restaurant,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Indian Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Gas Station,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Bank,Hakka Restaurant,Dumpling Restaurant,Drugstore
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,Spa,Convenience Store,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Department Store,Bus Station,Coffee Shop,Discount Store,Convenience Store,Chinese Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Dog Run
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Bakery,Bus Line,Soccer Field,Metro Station,Bus Station,Ice Cream Shop,Intersection,Park,Drugstore,Donut Shop
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,American Restaurant,Motel,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [19]:
north_york=toronto_merged.loc[toronto_merged['BOROUGH']=='North York']
north_york

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,Mediterranean Restaurant,Golf Course,Dog Run,Pool,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Japanese Restaurant,Toy / Game Store,Juice Bar,Bakery,Tea Room,Liquor Store
19,M2K,North York,Bayview Village,43.786947,-79.385975,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
20,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,Cafeteria,Deli / Bodega,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,Piano Bar,Deli / Bodega,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
22,M2N,North York,Willowdale South,43.770120,-79.408493,Ramen Restaurant,Café,Restaurant,Sushi Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Discount Store,Indonesian Restaurant,Electronics Store
23,M2P,North York,York Mills West,43.752758,-79.400049,Park,Bank,Bar,Convenience Store,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
24,M2R,North York,Willowdale West,43.782736,-79.442259,Pharmacy,Coffee Shop,Discount Store,Pizza Place,Grocery Store,Airport Food Court,Farmers Market,Event Space,Ethiopian Restaurant,Empanada Restaurant
25,M3A,North York,Parkwoods,43.753259,-79.329656,Park,Food & Drink Shop,Construction & Landscaping,Bus Stop,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
26,M3B,North York,Don Mills North,43.745906,-79.352188,Café,Caribbean Restaurant,Japanese Restaurant,Gym / Fitness Center,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore


In [20]:
north_york=north_york[['BOROUGH','NEIGHBORHOOD','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']]
north_york

,BOROUGH,NEIGHBORHOOD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
17,North York,Hillcrest Village,Mediterranean Restaurant,Golf Course,Dog Run
18,North York,"Fairview, Henry Farm, Oriole",Clothing Store,Fast Food Restaurant,Coffee Shop
19,North York,Bayview Village,Chinese Restaurant,Café,Bank
20,North York,"Silver Hills, York Mills",Cafeteria,Deli / Bodega,Ethiopian Restaurant
21,North York,"Newtonbrook, Willowdale",Piano Bar,Deli / Bodega,Ethiopian Restaurant
22,North York,Willowdale South,Ramen Restaurant,Café,Restaurant
23,North York,York Mills West,Park,Bank,Bar
24,North York,Willowdale West,Pharmacy,Coffee Shop,Discount Store
25,North York,Parkwoods,Park,Food & Drink Shop,Construction & Landscaping
26,North York,Don Mills North,Café,Caribbean Restaurant,Japanese Restaurant


In [21]:
fre1=pd.value_counts(north_york['1st Most Common Venue'])
fre1

Park                        4
Grocery Store               2
Coffee Shop                 2
Clothing Store              2
Asian Restaurant            1
Café                        1
Miscellaneous Shop          1
Baseball Field              1
Piano Bar                   1
Food Truck                  1
Italian Restaurant          1
Cafeteria                   1
Pharmacy                    1
Pizza Place                 1
Ramen Restaurant            1
Chinese Restaurant          1
Mediterranean Restaurant    1
Gym                         1
Name: 1st Most Common Venue, dtype: int64

In [22]:
fre2=pd.value_counts(north_york['2nd Most Common Venue'])
fre2

Coffee Shop                  4
Café                         2
Bank                         2
Deli / Bodega                2
Home Service                 1
Empanada Restaurant          1
Gym / Fitness Center         1
Airport                      1
Caribbean Restaurant         1
Food & Drink Shop            1
Fast Food Restaurant         1
Accessories Store            1
Japanese Restaurant          1
Bakery                       1
Middle Eastern Restaurant    1
Golf Course                  1
Women's Store                1
Pizza Place                  1
Name: 2nd Most Common Venue, dtype: int64

In [23]:
downtown_toronto=toronto_merged.loc[toronto_merged['BOROUGH']=='Downtown Toronto']
downtown_toronto

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,Park,Playground,Trail,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
51,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,Coffee Shop,Pizza Place,Italian Restaurant,Park,Café,Pub,Bakery,Restaurant,Hot Dog Joint,Playground
52,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,Coffee Shop,Japanese Restaurant,Restaurant,Gay Bar,Sushi Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gastropub
53,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Mexican Restaurant,Farmers Market,Event Space
54,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Bakery,Fast Food Restaurant,Middle Eastern Restaurant,Italian Restaurant,Restaurant
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Italian Restaurant,Bakery,Beer Bar,Cosmetics Shop,Breakfast Spot
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Steakhouse,Beer Bar,Bakery,Café,Cheese Shop,Eastern European Restaurant
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,Coffee Shop,Café,Ice Cream Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Burger Joint,Chinese Restaurant,Juice Bar,Salad Place
58,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,Coffee Shop,Café,Steakhouse,Bar,Asian Restaurant,Restaurant,Breakfast Spot,Hotel,Thai Restaurant,Gastropub
59,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Scenic Lookout,Restaurant,Brewery,Fried Chicken Joint,Music Venue


In [24]:
fre1=pd.value_counts(downtown_toronto['1st Most Common Venue'])
fre1

Coffee Shop       14
Café               2
Grocery Store      1
Park               1
Airport Lounge     1
Name: 1st Most Common Venue, dtype: int64

In [25]:
fre2=pd.value_counts(downtown_toronto['2nd Most Common Venue'])
fre2

Café                     8
Japanese Restaurant      2
Clothing Store           1
Cocktail Bar             1
Park                     1
Vietnamese Restaurant    1
Airport Service          1
Gym                      1
Pizza Place              1
Playground               1
Aquarium                 1
Name: 2nd Most Common Venue, dtype: int64

In [26]:
scarborough=toronto_merged.loc[toronto_merged['BOROUGH']=='Scarborough']
scarborough

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Fast Food Restaurant,Department Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Bar,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Intersection,Electronics Store,Rental Car Location,Pizza Place,Breakfast Spot,Mexican Restaurant,Medical Center,Doner Restaurant,Diner,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Indian Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Gas Station,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Bank,Hakka Restaurant,Dumpling Restaurant,Drugstore
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,Spa,Convenience Store,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Department Store,Bus Station,Coffee Shop,Discount Store,Convenience Store,Chinese Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Dog Run
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,Bakery,Bus Line,Soccer Field,Metro Station,Bus Station,Ice Cream Shop,Intersection,Park,Drugstore,Donut Shop
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,American Restaurant,Motel,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [27]:
fre1=pd.value_counts(scarborough['1st Most Common Venue'])
fre1

Fast Food Restaurant         2
Middle Eastern Restaurant    1
Bakery                       1
Coffee Shop                  1
Gas Station                  1
Pizza Place                  1
Department Store             1
Latin American Restaurant    1
Park                         1
Spa                          1
Intersection                 1
College Stadium              1
American Restaurant          1
Bar                          1
Indian Restaurant            1
Name: 1st Most Common Venue, dtype: int64

In [28]:
fre2=pd.value_counts(scarborough['2nd Most Common Venue'])
fre2

Convenience Store     2
Korean Restaurant     1
Bus Station           1
Chinese Restaurant    1
Bakery                1
Women's Store         1
Motel                 1
Department Store      1
Playground            1
Lounge                1
Auto Garage           1
Electronics Store     1
Bus Line              1
Café                  1
Pet Store             1
Name: 2nd Most Common Venue, dtype: int64

In [29]:
etobicoke=toronto_merged.loc[toronto_merged['BOROUGH']=='Etobicoke']
etobicoke

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,Café,Coffee Shop,Gym,Fast Food Restaurant,Restaurant,Fried Chicken Joint,Mexican Restaurant,Bakery,Pizza Place,Pharmacy
89,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,Pizza Place,Gym,Coffee Shop,Pharmacy,Skating Rink,Athletics & Sports,Pub,Sandwich Place,Discount Store,Dessert Shop
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,River,Deli / Bodega,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509,Breakfast Spot,Baseball Field,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
92,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999,Gym,Bakery,Convenience Store,Discount Store,Burrito Place,Burger Joint,Sandwich Place,Flower Shop,Supplement Shop,Fast Food Restaurant
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201,Café,Coffee Shop,Cosmetics Shop,Beer Store,Convenience Store,Pizza Place,Liquor Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant
99,M9P,Etobicoke,Westmount,43.696319,-79.532242,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Discount Store,Intersection,Chinese Restaurant,Women's Store,Dog Run,Dim Sum Restaurant
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724,Mobile Phone Shop,Bus Line,Pizza Place,Sandwich Place,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437,Grocery Store,Coffee Shop,Pizza Place,Fried Chicken Joint,Sandwich Place,Beer Store,Fast Food Restaurant,Video Store,Pharmacy,Liquor Store


In [30]:
fre1=pd.value_counts(etobicoke['1st Most Common Venue'])
fre1

Café                   2
Pizza Place            2
Grocery Store          1
Rental Car Location    1
Breakfast Spot         1
Mobile Phone Shop      1
River                  1
Gym                    1
Name: 1st Most Common Venue, dtype: int64

In [31]:
fre2=pd.value_counts(etobicoke['2nd Most Common Venue'])
fre2

Coffee Shop       4
Bus Line          1
Bakery            1
Drugstore         1
Baseball Field    1
Deli / Bodega     1
Gym               1
Name: 2nd Most Common Venue, dtype: int64

In [32]:
central_toronto=toronto_merged.loc[toronto_merged['BOROUGH']=='Central Toronto']
central_toronto

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,Park,Bus Line,Swim School,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,Gym,Hotel,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,General Entertainment,Electronics Store,Gift Shop
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,Coffee Shop,Sporting Goods Shop,Yoga Studio,Mexican Restaurant,Rental Car Location,Dessert Shop,Salon / Barbershop,Café,Diner,Fast Food Restaurant
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Pizza Place,Coffee Shop,Sushi Restaurant,Gym,Diner,Farmers Market
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,Gym,Playground,Tennis Court,College Stadium,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,Pub,Coffee Shop,Sushi Restaurant,Light Rail Station,Vietnamese Restaurant,Restaurant,Liquor Store,Supermarket,Pizza Place,American Restaurant
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936,Garden,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Deli / Bodega
64,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,Jewelry Store,Mexican Restaurant,Sushi Restaurant,Trail,Women's Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,Café,Sandwich Place,Coffee Shop,Pizza Place,Middle Eastern Restaurant,Flower Shop,Pub,Indian Restaurant,BBQ Joint,History Museum


In [33]:
fre1=pd.value_counts(central_toronto['1st Most Common Venue'])
fre1

Gym              2
Café             1
Pub              1
Garden           1
Park             1
Coffee Shop      1
Dessert Shop     1
Jewelry Store    1
Name: 1st Most Common Venue, dtype: int64

In [34]:
fre2=pd.value_counts(central_toronto['2nd Most Common Venue'])
fre2

Sandwich Place         2
Mexican Restaurant     1
Women's Store          1
Bus Line               1
Coffee Shop            1
Sporting Goods Shop    1
Hotel                  1
Playground             1
Name: 2nd Most Common Venue, dtype: int64

In [35]:
west_toronto=toronto_merged.loc[toronto_merged['BOROUGH']=='West Toronto']
west_toronto

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,Pharmacy,Bakery,Bar,Park,Grocery Store,Bank,Gym / Fitness Center,Brewery,Middle Eastern Restaurant,Music Venue
77,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,Bar,Coffee Shop,Men's Store,Restaurant,Vietnamese Restaurant,Pizza Place,Café,Asian Restaurant,Yoga Studio,Juice Bar
78,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,Coffee Shop,Breakfast Spot,Café,Bakery,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
82,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,Café,Bar,Mexican Restaurant,Thai Restaurant,Italian Restaurant,Flea Market,Bakery,Diner,Fried Chicken Joint,Speakeasy
83,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,Gift Shop,Italian Restaurant,Cuban Restaurant,Restaurant,Bookstore,Bar,Dog Run,Movie Theater,Dessert Shop,Eastern European Restaurant
84,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,Pizza Place,Café,Coffee Shop,Tea Room,Italian Restaurant,Sushi Restaurant,Diner,Bar,Dessert Shop,Fish & Chips Shop


In [36]:
fre1=pd.value_counts(west_toronto['1st Most Common Venue'])
fre1

Coffee Shop    1
Gift Shop      1
Café           1
Bar            1
Pharmacy       1
Pizza Place    1
Name: 1st Most Common Venue, dtype: int64

In [37]:
fre2=pd.value_counts(west_toronto['2nd Most Common Venue'])
fre2

Italian Restaurant    1
Coffee Shop           1
Café                  1
Breakfast Spot        1
Bar                   1
Bakery                1
Name: 2nd Most Common Venue, dtype: int64

In [38]:
york=toronto_merged.loc[toronto_merged['BOROUGH']=='York']
york

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,Trail,Park,Field,Hockey Arena,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
74,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,Park,Women's Store,Fast Food Restaurant,Market,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
80,M6M,York,"Del Ray, Keelesdale, Mount Dennis, Silverthorn",43.691116,-79.476013,Turkish Restaurant,Coffee Shop,Skating Rink,Bar,Restaurant,Sandwich Place,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner
81,M6N,York,"The Junction North, Runnymede",43.673185,-79.487262,Brewery,Pizza Place,Breakfast Spot,Convenience Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
98,M9N,York,Weston,43.706876,-79.518188,Park,Convenience Store,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [39]:
fre1=pd.value_counts(york['1st Most Common Venue'])
fre1

Park                  2
Trail                 1
Turkish Restaurant    1
Brewery               1
Name: 1st Most Common Venue, dtype: int64

In [40]:
fre2=pd.value_counts(york['2nd Most Common Venue'])
fre2

Coffee Shop          1
Women's Store        1
Pizza Place          1
Park                 1
Convenience Store    1
Name: 2nd Most Common Venue, dtype: int64

In [41]:
east_york=toronto_merged.loc[toronto_merged['BOROUGH']=='East York']
east_york

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,Pizza Place,Fast Food Restaurant,Pet Store,Pharmacy,Gym / Fitness Center,Gastropub,Bank,Bus Line,Athletics & Sports,Intersection
36,M4C,East York,Woodbine Heights,43.695344,-79.318389,Pharmacy,Video Store,Cosmetics Shop,Curling Ice,Skating Rink,Beer Store,Park,General Entertainment,Gift Shop,Eastern European Restaurant
38,M4G,East York,Leaside,43.709060,-79.363452,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Sandwich Place,Burger Joint,Restaurant,Sushi Restaurant,Supermarket,Fish & Chips Shop,Liquor Store
39,M4H,East York,Thorncliffe Park,43.705369,-79.349372,Indian Restaurant,Gym,Supermarket,Bank,Burger Joint,Coffee Shop,Fast Food Restaurant,Gas Station,Grocery Store,Gym / Fitness Center
40,M4J,East York,East Toronto,43.685347,-79.338106,Park,Convenience Store,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [42]:
fre1=pd.value_counts(east_york['1st Most Common Venue'])
fre1

Coffee Shop          1
Pizza Place          1
Pharmacy             1
Park                 1
Indian Restaurant    1
Name: 1st Most Common Venue, dtype: int64

In [43]:
fre2=pd.value_counts(east_york['2nd Most Common Venue'])
fre2

Fast Food Restaurant    1
Sporting Goods Shop     1
Gym                     1
Video Store             1
Convenience Store       1
Name: 2nd Most Common Venue, dtype: int64

In [44]:
east_toronto=toronto_merged.loc[toronto_merged['BOROUGH']=='East Toronto']
east_toronto

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,Park,Trail,Pub,Health Food Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dance Studio
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Diner,Caribbean Restaurant,Dessert Shop,Pub
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,Park,Pet Store,Board Shop,Brewery,Sandwich Place,Burger Joint,Burrito Place,Pub,Gym,Sushi Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,Café,Coffee Shop,Bakery,Brewery,Gastropub,Italian Restaurant,American Restaurant,Yoga Studio,Bar,Fish Market
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Yoga Studio,Auto Workshop,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Recording Studio


In [45]:
fre1=pd.value_counts(east_toronto['1st Most Common Venue'])
fre1

Park                2
Yoga Studio         1
Greek Restaurant    1
Café                1
Name: 1st Most Common Venue, dtype: int64

In [46]:
fre2=pd.value_counts(east_toronto['2nd Most Common Venue'])
fre2

Coffee Shop      2
Auto Workshop    1
Trail            1
Pet Store        1
Name: 2nd Most Common Venue, dtype: int64

In [47]:
mississauga=toronto_merged.loc[toronto_merged['BOROUGH']=='Mississauga']
mississauga

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819,Coffee Shop,Hotel,Gym,Mediterranean Restaurant,Fried Chicken Joint,Middle Eastern Restaurant,Burrito Place,Sandwich Place,American Restaurant,Donut Shop


In [48]:
queens_park=toronto_merged.loc[toronto_merged['BOROUGH']=="Queen's Park"]
queens_park

,POSTALCODE,BOROUGH,NEIGHBORHOOD,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,M9A,Queen's Park,Queen's Park,43.667856,-79.532242,Coffee Shop,Gym,Park,Fast Food Restaurant,Salad Place,Portuguese Restaurant,Nightclub,Music Venue,Mexican Restaurant,Juice Bar
